## 1. 사용할 패키지 불러오기

In [25]:
import numpy as np
import pandas as pd
import os
import keras
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from fastprogress import master_bar, progress_bar
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

## 2. 데이터 불러오기

In [26]:
data = pd.read_csv("data.csv")
data.head()

,시가(포인트),고가(포인트),저가(포인트),종가(포인트),종가베이시스,코스피200시가총액,거래량(계약),거래대금(천원),잔존일수(일),미결제약정(계약),...,국가별수출액 합계(천달러),WTI선물_종가,브렌트유선물_종가,천연가스선물_종가,금선물_종가,은선물_종가,구리선물_종가,MACD,RSI,Date
0,238.30,239.25,235.60,238.30,1.58,7.939651e+14,154511.0,1.836201e+10,88.0,79906.0,...,30998134.0,80.570000,76.980003,6.653,715.799988,12.739,3.4175,0.828727,52.630885,2007-09-17
1,238.00,238.60,233.15,233.60,1.18,7.794642e+14,199834.0,2.359650e+10,87.0,79909.0,...,30998134.0,81.510002,77.589996,6.568,715.799988,12.767,3.4425,0.724943,47.740409,2007-09-18
2,242.55,245.00,241.45,245.00,3.92,8.076760e+14,162505.0,1.973762e+10,86.0,84491.0,...,30998134.0,81.930000,78.470001,6.180,722.000000,12.956,3.5675,1.544772,57.947348,2007-09-19
3,244.00,246.35,243.25,244.65,2.23,8.119758e+14,170882.0,2.089486e+10,85.0,84638.0,...,30998134.0,83.320000,79.089996,6.008,732.400024,13.321,3.5740,2.141563,57.575543,2007-09-20
4,244.65,247.10,244.25,245.80,2.35,8.155575e+14,150500.0,1.847012e+10,84.0,81140.0,...,30998134.0,81.620003,79.300003,6.080,731.400024,13.474,3.5725,2.676467,58.517352,2007-09-21


## 3. 데이터 전처리

#### - 증감 여부 추가

In [27]:
data['Increasing'] = 0
for i in range(1, len(data)):
    if data['종가(포인트)'][i] > data['종가(포인트)'][i-1]:
        data['Increasing'][i] = 1


/tmp/ipykernel_11029/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Increasing'][i] = 1
/tmp/ipykernel_11029/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Increasing'][i] = 1
/tmp/ipykernel_11029/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Increasing'][i] = 1
/tmp/ipykernel_11029/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

#### - Input 과 Target 정의

In [28]:
Input_columns = ['시가(포인트)', '고가(포인트)', '저가(포인트)', '종가베이시스', '코스피200시가총액',
       '거래량(계약)', '거래대금(천원)', '잔존일수(일)', '미결제약정(계약)',
       '주가지수선물-Futures(SP합)-순매수대금(개인)(백만원)',
       '주가지수선물-Futures(SP합)-순매수대금(외국인)(백만원)',
       '주가지수선물-Futures(SP합)-순매수대금(기관계)(백만원)', '시장금리:CD유통수익률(91)(%)',
       '시장평균_미국(달러)(통화대원)', '소비자물가지수 총지수(2020=100)', '생산자물가지수 총지수(2015=100)',
       '전산업생산지수(2020=100)', '국가별수출액 합계(천달러)', 'WTI선물_종가', '브렌트유선물_종가',
       '천연가스선물_종가', '금선물_종가', '은선물_종가', '구리선물_종가', 'MACD', 'RSI', 'Increasing']
Target_column = 'Increasing'

### (1) Min-Max 정규화

In [29]:
print("Min-Max Normalization method is selected.")
Scalar_transform = MinMaxScaler(feature_range=(-1, 1)).fit(data.loc[:, [Target_column]])
data.loc[:, [Target_column]] = Scalar_transform.transform(data.loc[:, [Target_column]])
Input_transform = MinMaxScaler().fit(data.loc[:, Input_columns])
data.loc[:, Input_columns] = Input_transform.transform(data.loc[:, Input_columns])

Min-Max Normalization method is selected.


### (2) Transform Dataset with Sliding Windows

In [30]:
def sliding_windows(data, Input_columns, Target_column, seq_length):
    x = []
    y = []

    data_y = np.array(data.loc[:, [Target_column]])
    data_x = np.array(data.loc[:, Input_columns])


    for i in range(len(data_y)-seq_length):
        _x = data_x[i:(i+seq_length)]
        _y = data_y[i+seq_length]
        x.append(_x)
        y.append(_y)

    # x = np.array(x)
    return np.array(x),np.array(y)

seq_length = 60
x, y = sliding_windows(data, Input_columns, Target_column, seq_length)
x = x.reshape((x.shape[0], x.shape[1] * x.shape[2]))
print(x.shape)
print(y.shape)

(3761, 1620)
(3761, 1)


### (3) PCA

In [55]:
pca = PCA(n_components=1620)
pca_transformed = pca.fit_transform(x)
pca_results = pd.DataFrame({"PC": range(1,1621),
                            "설명가능한 분산 비율": pca.explained_variance_,
                            "기여율": pca.explained_variance_ratio_})
pca_results['누적기여율'] = pca_results['기여율'].cumsum()
pca_results

,PC,설명가능한 분산 비율,기여율,누적기여율
0,1,2.262894e+01,3.460564e-01,0.346056
1,2,7.993424e+00,1.222406e-01,0.468297
2,3,3.187211e+00,4.874089e-02,0.517038
3,4,2.344443e+00,3.585275e-02,0.552891
4,5,1.597253e+00,2.442623e-02,0.577317
...,...,...,...,...
1615,1616,5.380708e-08,8.228529e-10,1.000000
1616,1617,5.076737e-08,7.763676e-10,1.000000
1617,1618,4.939657e-08,7.554045e-10,1.000000
1618,1619,4.476648e-08,6.845980e-10,1.000000


In [56]:
pca_results.loc[pca_results['누적기여율'] >= 0.85, :]

,PC,설명가능한 분산 비율,기여율,누적기여율
52,53,2.438287e-01,3.728786e-03,0.850146
53,54,2.419912e-01,3.700686e-03,0.853847
54,55,2.405392e-01,3.678482e-03,0.857525
55,56,2.397629e-01,3.666610e-03,0.861192
56,57,2.377401e-01,3.635676e-03,0.864828
...,...,...,...,...
1615,1616,5.380708e-08,8.228529e-10,1.000000
1616,1617,5.076737e-08,7.763676e-10,1.000000
1617,1618,4.939657e-08,7.554045e-10,1.000000
1618,1619,4.476648e-08,6.845980e-10,1.000000


In [57]:
N_PCs = 52

In [58]:
pca_transformed = pca_transformed[:, :N_PCs]


### (4) Train Test Split

In [69]:
N_test = 300
train_x = pca_transformed[:(-N_test), :]
train_y = y[:-N_test, 0]
test_x = pca_transformed[(-N_test):, :]
test_y = y[-N_test:, 0]

## 4. Modeling

In [71]:
n_estimators_candidate = [100, 300,500]
max_depth_candidate = [3, 5, 7]
learning_rate_candidate = [0.1, 0.001]

# 결과를 저장할 빈 리스트 생성
n_estimators_list = []
max_depth_list = []
learning_rate_list = []
train_score_list = []
val_score_list = []

# XGBoost의 n_estimators 파라미터에 대해서
for n_estimators in n_estimators_candidate:
    # XGBoost의 max_depth 파라미터에 대해서
    for max_depth in max_depth_candidate:
        # XGBoost의 learning_rate 파라미터에 대해서
        for learning_rate in learning_rate_candidate:
            # 모델 생성 및 학습
            model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, objective='reg:squarederror').fit(train_x, train_y)
            # Train 데이터에 대한 결과 확인
            train_predict = model.predict(train_x)
            train_score_list.append(accuracy_score(train_y, train_predict))
            # Test 데이터에 대한 결과 확인
            test_predict = model.predict(test_x)
            val_score_list.append(accuracy_score(test_y, test_predict))
            # Parameter 저장
            n_estimators_list.append(n_estimators)
            max_depth_list.append(max_depth)
            learning_rate_list.append(learning_rate)
            
result = pd.DataFrame({"n_estimators": n_estimators_list, "max_depth": max_depth_list, "learning_rate": learning_rate_list, 'Train Score': train_score_list, 'Test Score': val_score_list})
result                    

,n_estimators,max_depth,learning_rate,Train Score,Test Score
0,100,3,0.100,0.809882,0.520000
1,100,3,0.001,0.539439,0.506667
2,100,5,0.100,0.978041,0.500000
3,100,5,0.001,0.548685,0.506667
4,100,7,0.100,1.000000,0.540000
5,100,7,0.001,0.579312,0.500000
6,300,3,0.100,0.952326,0.526667
7,300,3,0.001,0.544351,0.506667
8,300,5,0.100,1.000000,0.523333
9,300,5,0.001,0.573534,0.506667


In [72]:
result = result.loc[result['Test Score'] == max(result['Test Score']), :].reset_index(drop = True)
xgb_model = XGBClassifier(n_estimators=result['n_estimators'][0], max_depth=result['max_depth'][0], learning_rate=result['learning_rate'][0], objective='reg:squarederror').fit(train_x, train_y)
result

,n_estimators,max_depth,learning_rate,Train Score,Test Score
0,100,7,0.1,1.0,0.54


## 5. 성능평가

In [73]:
test_predict = xgb_model.predict(test_x)
print("Accuracy: {}".format(accuracy_score(test_y, test_predict)))

Accuracy: 0.54


## 6. 미래 예측

In [77]:
future_x = np.array(data.loc[:, Input_columns])[-seq_length:]
future_x = future_x.reshape((1, future_x.shape[0] * future_x.shape[1]))
future_x = pca.transform(future_x)
future_x = future_x[:, :N_PCs]

In [78]:
if xgb_model.predict(future_x) > 0.5:
    print("Increase")
else:
    print("Decrease")

Increase
